<a href="https://colab.research.google.com/github/TheLittleJimmy/CBB750/blob/main/EDA_larger_SDM_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
data_path='/content/drive/MyDrive/BIS550_CBB750/project/Code by message type.xlsx'

In [3]:
import pandas as pd

In [4]:
IG_P = pd.read_excel(data_path, sheet_name='Info Giving - Patient')
IG_C = pd.read_excel(data_path, sheet_name='Info Giving - Clinician')
IG_A = pd.read_excel(data_path, sheet_name='Info Giving - Auto')

IS_C = pd.read_excel(data_path, sheet_name='Info Seek - Clinician')
IS_P = pd.read_excel(data_path, sheet_name='Info Seek - Patient')

Emo_P = pd.read_excel(data_path, sheet_name='Emotion - Patient')
Emo_C = pd.read_excel(data_path, sheet_name='Emotion - Clinician')

P_C = pd.read_excel(data_path, sheet_name='Partnership - Clinician')
P_P = pd.read_excel(data_path, sheet_name='Partnership - Patient')

SDM_P = pd.read_excel(data_path, sheet_name='SDM - Patient')
SDM_C = pd.read_excel(data_path, sheet_name='SDM - Clinician',usecols=[0,1])

In [5]:
# Combine IG_P, IG_C, IG_A into one dataframe
IG = pd.concat([IG_P, IG_C, IG_A], axis=0, ignore_index=True)
# Combine IS_C, IS_P into one dataframe
IS = pd.concat([IS_C, IS_P], axis=0, ignore_index=True)
# Combine Emo_P, Emo_C into one dataframe
Emo = pd.concat([Emo_P, Emo_C], axis=0, ignore_index=True)
# Combine P_C, P_P into one dataframe
P = pd.concat([P_C, P_P], axis=0, ignore_index=True)
# Combine SDM_C, SDM_P into one dataframe
SDM = pd.concat([SDM_C, SDM_P], axis=0, ignore_index=True) # SDM - shared decision making # really small class, need data augmentation

In [36]:
SDM

,Quotation Content,Codes,Label
0,Hi That sounds reasonable I would do the colon...,Clinician,SDM
1,Hi Mr It is most likely a radiation side effec...,Clinician,SDM
2,Sounds good- I just put in prescription for- n...,Clinician,SDM
3,Dr I have not scheduled the colonoscopy as yet...,Patient,SDM
4,GH and Comprehensive Metabolic panel blood tes...,Patient,SDM


In [6]:
# Add labels to each dataframe
IG['Label'] = 'Info Giving'
IS['Label'] = 'Info Seeking'
Emo['Label'] = 'Emotion'
P['Label'] = 'Partnership'
SDM['Label'] = 'SDM'
# Combine all dataframes into one dataframe
df = pd.concat([IG, IS, Emo, P, SDM], axis=0, ignore_index=True)
df

,Quotation Content,Codes,Label
0,197 I noticed that my medication list has not ...,Patient,Info Giving
1,Blood in urine and stool- began yesterday We a...,Patient,Info Giving
2,Can someone set up that appointment as I do no...,Patient,Info Giving
3,Dear Doctor Lacy I just was told my daughter w...,Patient,Info Giving
4,Dear Dr Chiang When I saw you last Thursday( M...,Patient,Info Giving
...,...,...,...
270,Hi That sounds reasonable I would do the colon...,Clinician,SDM
271,Hi Mr It is most likely a radiation side effec...,Clinician,SDM
272,Sounds good- I just put in prescription for- n...,Clinician,SDM
273,Dr I have not scheduled the colonoscopy as yet...,Patient,SDM


In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Easy Data Augmentation (EDA)
Four operations:  
Synonym Replacement (SR): Randomly choose n words from the sentence that are not stop words. Replace each of these words with one of its synonyms chosen at random.  
Random Insertion (RI): Find a random synonym of a random word in the sentence that is not a stop word. Insert that synonym into a random position in the sentence. Do this n times.  
Random Swap (RS): Randomly choose two words in the sentence and swap their positions. Do this n times.  
Random Deletion (RD): Randomly remove each word in the sentence with probability p. 

For more details see github repo: https://github.com/jasonwei20/eda_nlp

default parameters:
num_aug (the number of augmented sentences to generate given a sentense) = 9  
alpha parameters (percent of words in the sentence that will be changed) = 0.1  
alpha parameters include alpha_sr, alpha_rd, alpha_ri, alpha_rs

In [31]:
# put messages labeled SDM to a file in the format label\tsentence
f=open("sdm.txt","w")
for msg in SDM["Quotation Content"]:
  f.write(f"0\t{msg.strip()}\n")
f.close()

In [33]:
# perform eda using default parameters
%%python augment.py --input=sdm.txt

generated augmented sentences with eda for sdm.txt to eda_sdm.txt with num_aug=9


In [37]:
# take a look at the output and save augmented sentences to a list
eda_sdm=[]
f=open("eda_sdm.txt","r")
for eda_msg in f:
  print(eda_msg.strip())
  eda_sdm.append(eda_msg.strip().split("\t")[1])
f.close()

0	hi that auditory sensation reasonable i would do the colonoscopy more electively when no pressing issues are noted best vik
0	hi that sounds reasonable i would do expert the colonoscopy more electively when no pressing issues are noted best vik
0	hi that sounds reasonable i would do the colonoscopy more electively when no pressing issues event are noted best vik
0	hi that sounds reasonable i would do the colonoscopy more electively when no pressing issues are noted best vik
0	hi that sounds reasonable i would do when colonoscopy more electively the no pressing issues are noted best vik
0	hi that sounds reasonable i would do the colonoscopy more electively when no pressing issues are noted honest vik
0	hi that sounds reasonable i would do issues colonoscopy more electively when no pressing the are noted best vik
0	hi that sounds reasonable i would do the colonoscopy more electively when no pressing issues are how do you do noted best vik
0	hi that sounds reasonable i would do the colo

In [41]:
EDA_SDM=pd.DataFrame(data=zip(*[eda_sdm,["SDM" for i in range(len(eda_sdm))]]),columns=["Quotation Content","Label"])

In [42]:
EDA_SDM

,Quotation Content,Label
0,hi that auditory sensation reasonable i would ...,SDM
1,hi that sounds reasonable i would do expert th...,SDM
2,hi that sounds reasonable i would do the colon...,SDM
3,hi that sounds reasonable i would do the colon...,SDM
4,hi that sounds reasonable i would do when colo...,SDM
5,hi that sounds reasonable i would do the colon...,SDM
6,hi that sounds reasonable i would do issues co...,SDM
7,hi that sounds reasonable i would do the colon...,SDM
8,hi that sounds reasonable i would do the colon...,SDM
9,hi that sounds reasonable i would do the colon...,SDM


TODO:
Sequence length

Feature extraction

Wordnet vs UMLS